In [1]:
import json
import pickle
import random
#import argparse
import numpy as np
import jieba

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F

In [2]:
def read_data_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

def write_data_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
        
def read_json_line(filename):
    data_list = []
    with open(filename, 'r') as f:
        for line in f:
            data_list.append(json.loads(line))
    return data_list

def read_math23k_json(filename):
    data_list = []
    with open(filename, 'r') as f:
        count = 0
        string = ''
        for line in f:
            count += 1
            string += line
            if count % 7 == 0:
                #print string
                data_list.append(json.loads(string))
                string = ''
    #print data_list[-1]
    return data_list

In [3]:
math23k_test = read_math23k_json("./data/math23k_test.json")
math23k_train = read_math23k_json("./data/math23k_train.json")

In [4]:
for elem in math23k_train:
    origin = elem['original_text']
    origin_text = ' '.join(jieba.cut(origin, cut_all=False))
    elem['new_split'] = origin_text
for elem in math23k_test:
    origin = elem['original_text']
    origin_text = ' '.join(jieba.cut(origin, cut_all=False))
    elem['new_split'] = origin_text

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.895 seconds.
Prefix dict has been built succesfully.


In [5]:
def joint_number(text_list):
    new_list = []
    i = 0
    while i<len(text_list):
        if text_list[i] == '(' and i+4<len(text_list) and text_list[i+4] == ')':
            sub = ''.join(text_list[i:i+5])
            new_list.append(sub)
            i = i+5
        else:
            new_list.append(text_list[i])
            i += 1
    #text_list = new_list[:]
    #new_list = []
    '''
    i = 0
    while i < len(text_list):
        if '%' in text_list[i] and len(text_list[i])>1:
            new_list.append(text_list[i])
            new_list.append(text_list[i][-1])
        else:
            new_list.append(text_list[i])
        i += 1
    '''
    return new_list        

def is_number(word):
    if word[0] == '(' and word[-1] == ')':
        for elem_char in word:
            if (elem_char.isdigit()):
                return True
        return False
    if '(' in word and ')' in word and '/' in word and not word[-1].isdigit():
        for elem_char in word:
            if (elem_char.isdigit()):
                return True
        return False
        #return True
    if word[-1] == '%' and len(word)>1:
        return True
    if word[0].isdigit():
        return True
    if word[-1].isdigit():
        return True
    try:
        float(word)
        return True
    except:
        return False
    
def split_num_and_unit(word):
    num = ''
    unit = ''
    for idx in range(len(word)):                                                                                                                                                                                                                                              
        char = word[idx]
        if char.isdigit() or char in ['.', '/', '(', ')']:
            num += char
        else:
            unit += char
    return num, unit#.encode('utf-8')

def mask_num(seg_text_list, equ_str):
    origin_equ_str = equ_str[:]

    alphas = 'abcdefghijklmnopqrstuvwxyz'
    num_list  = []
    mask_seg_text = []
    count = 0 
    number_pos = []
    num_temp_list = []
    for w_i in range(len(seg_text_list)):
        word = seg_text_list[w_i]
        if word == '':
            continue
        if is_number(word):
            mask_seg_text.append("temp_"+alphas[count])
            if '%' in word:
                mask_seg_text.append('%')
            #elif 'm' in word.lower() or 'g' in word.lower() or 'd' in word.lower():
            elif len(set(alphas)&set(word.lower()))>0:
                num, unit = split_num_and_unit(word)
                mask_seg_text.append(unit)
                word = num
                
            num_list.append(word)
            number_pos.append(w_i)
            num_temp_list.append("temp_"+alphas[count])
            count += 1
        else:
            mask_seg_text.append(word)
    mask_equ_list = []
    s_n = sorted([(w,i) for i,w in enumerate(num_list)], key=lambda x: len(str(x[0])), reverse=True)
    if '3.14%' not in equ_str and '3.1416' not in equ_str:
        equ_str = equ_str.replace('3.14', '&PI&', 15)
    new_equ_str = ''
    #print (s_n)
    #print (equ_str)
    for num, idx in s_n:
        #num = num_list[idx]
        equ_str = equ_str.replace(num, '&temp_'+alphas[idx]+'&', 15)
        #if 
    #print (equ_str)
        
        
    equ_list = []
    num_set = ['0','1','2','3','4','5','6','7','8','9','%', '.']
    for elem in equ_str.split('&'):
        if 'temp' in elem or 'PI' in elem:
            equ_list.append(elem)
        else:
            start = ''
            for char in elem:
                if char not in num_set:
                    if start != '':
                        equ_list.append(start)
                    equ_list.append(char)
                    start = ''
                else:
                    start += char
            if start != '':
                equ_list.append(start)
    #rint (equ_list)
    #rint ()
    #reverse_equ_list = equ_list[::-1]
    #reverse_equ_list.append('END_token')
    #equ_list.append('END_token')
    '''
    print (' '.join(seg_text_list))
    print (' '.join(mask_seg_text))
    print (num_list)
    print (' '.join(equ_list))
    print (origin_equ_str)
    print ()
    '''
    new_equ_list = []
    for elem_equ in equ_list:
        if elem_equ == '[':
            elem_equ = '('
        if elem_equ == ']':
            elem_equ = ')'
        new_equ_list.append(elem_equ)
    equ_list = new_equ_list[:]
    return mask_seg_text, num_list, equ_list, number_pos, num_temp_list



def num_list_processed(num_list):
    new_num_list = []
    for num in num_list:
        if '%' in num:
            new_num_list.append(float(num[:-1])*1.0/100)
        elif '(' in num:
            new_num_list.append(eval(num))
         
        else:
            num_,_ = split_num_and_unit(num)
            new_num_list.append(float(num_))
    return new_num_list

def postfix_equation(equ_list):
    stack = []
    post_equ = []
    op_list = ['+', '-', '*', '/', '^']
    priori = {'^':3, '*':2, '/':2, '+':1, '-':1}
    for elem in equ_list:
        if elem == '(':
            stack.append('(')
        elif elem == ')':
            while 1:
                op = stack.pop()
                if op == '(':
                    break
                else:
                    post_equ.append(op)
        elif elem in op_list:
            while 1:
                if stack == []:
                    break
                elif stack[-1] == '(':
                    break
                elif priori[elem] > priori[stack[-1]]:
                    break
                else:
                    op = stack.pop()
                    post_equ.append(op)
            stack.append(elem)
        else:
            #if elem == 'PI':
            #    post_equ.append('3.14')
            #else:
            #    post_equ.append(elem)
            post_equ.append(elem)
    while stack != []:
        post_equ.append(stack.pop())
    return post_equ

def post_solver(post_equ):
    stack = []
    op_list = ['+', '-', '/', '*', '^']
    for elem in post_equ:
        if elem not in op_list:
            op_v = elem
            #if '%' in op_v:
            #    op_v = float(op_v[:-1])/100.0
            stack.append(str(op_v))
        elif elem in op_list:
            op_v_1 = stack.pop()
            op_v_1 = float(op_v_1)
            op_v_2 = stack.pop()
            op_v_2 = float(op_v_2)
            if elem == '+':
                stack.append(str(op_v_2+op_v_1))
            elif elem == '-':
                stack.append(str(op_v_2-op_v_1))
            elif elem == '*':
                stack.append(str(op_v_2*op_v_1))
            elif elem == '/':
                stack.append(str(op_v_2/op_v_1))
            else:
                stack.append(str(op_v_2**op_v_1))
    return stack.pop()
         
def solve_equation(equ_list):
    if '=' in equ_list:
        equ_list = equ_list[2:]
   
    post_equ = postfix_equation(equ_list)
    ans = post_solver(post_equ)
    return ans

def inverse_temp_to_num(equ_list, num_list):
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    new_equ_list = []
    for elem in equ_list:
        if 'temp' in elem:
            index = alphabet.index(elem[-1])
            new_equ_list.append(str(num_list[index]))
        elif 'PI' == elem:
            new_equ_list.append('3.14')
        else:
            new_equ_list.append(elem)
    return new_equ_list

def ans_num_joint(word):
    i = 0
    new = []
    str_ = ''
    while i<len(word):
        if word[i].isdigit() or word[i] in ['.','-']:
            str_ += word[i]
        else:
            if str_ != '':
                new.append(str_)
                str_ = ''
            new.append(word[i])
        i+=1
    return solve_equation(new)

def ans_decimal_exception(word):
    word = str(word)
    ind = word.find('(')
    word = word[:ind]+'+'+word[ind:]
    return ans_num_joint(word)

def ans_process(word):
    try:
        float(word)
        return float(word)
    except:
        if '%' in str(word):
            return float(word[:-1])/100
        if str(word)[0]=='(' and str(word)[-1]==')':
            return ans_num_joint(word)
        if str(word)[0] != '(' and str(word)[-1]==')':
            return ans_decimal_exception(word)
    return -float('inf')

In [6]:
for elem in math23k_train:
    pid = elem['id']
    split_text = elem['new_split']
    text_list = split_text.split(' ')
    join_text_list = joint_number(text_list) # '(' '1' '/' '5' ')' -> '(1/5)'
    equation_str = elem['equation']
    mask_seg_text, num_list, temp_equ_list, num_pos, num_temp_list = mask_num(join_text_list, equation_str)
    #print (mask_seg_text)
    #print (split_text)
    #print (num_list)
    #print ([mask_seg_text[pos] for pos in num_pos] )
    #print (num_pos)
    elem['mask_seg_text'] = mask_seg_text
    context_list = []
    for pos in num_pos:
        if pos - 3<0:
            left = 0
        else:
            left = pos - 3
        if pos + 4 >= len(mask_seg_text):
            right = len(mask_seg_text)
        else:
            right = pos + 4
        context_list.append(mask_seg_text[left: right])
    elem['context_list'] = context_list[:]    
    
    num_labels = []
    for temp_num in num_temp_list:
        if temp_num not in temp_equ_list:
            num_labels.append(0)
        else:
            num_labels.append(1)
    elem['num_labels'] = num_labels
    assert len(num_labels) == len(context_list), "alignment error"
    #if 0 in num_labels:
        #pass
        #print (num_labels)
        #print (num_temp_list)
        #print (elem['context_list'])
        #print (temp_equ_list)
        #print ()
    #break

In [7]:
for elem in math23k_test:
    pid = elem['id']
    split_text = elem['new_split']
    text_list = split_text.split(' ')
    join_text_list = joint_number(text_list) # '(' '1' '/' '5' ')' -> '(1/5)'
    equation_str = elem['equation']
    mask_seg_text, num_list, temp_equ_list, num_pos, num_temp_list = mask_num(join_text_list, equation_str)
    #print (mask_seg_text)
    #print (split_text)
    #print (num_list)
    #print ([mask_seg_text[pos] for pos in num_pos] )
    #print (num_pos)
    elem['mask_seg_text'] = mask_seg_text
    context_list = []
    for pos in num_pos:
        if pos - 3<0:
            left = 0
        else:
            left = pos - 3
        if pos + 4 >= len(mask_seg_text):
            right = len(mask_seg_text)
        else:
            right = pos + 4
        context_list.append(mask_seg_text[left: right])
    elem['context_list'] = context_list[:]    
    
    num_labels = []
    for temp_num in num_temp_list:
        if temp_num not in temp_equ_list:
            num_labels.append(0)
        else:
            num_labels.append(1)
    elem['num_labels'] = num_labels
    assert len(num_labels) == len(context_list), "alignment error"
    #if 0 in num_labels:
        #pass
        #print (num_labels)
        #print (num_temp_list)
        #print (elem['context_list'])
        #print (temp_equ_list)
        #print ()
    #break

In [8]:
def string_2_idx_sen(sen,  vocab_dict):                                                                                             
    new = []
    for word in sen:
        if word not in vocab_dict:
             new.append(vocab_dict['UNK_token'])
        else:
             new.append(vocab_dict[word])
    return new

def pad_sen(sen_idx_list, max_len=7, pad_idx=0):                                                                                                                                                                                                                            
    return [pad_idx]*(max_len-len(sen_idx_list))+ sen_idx_list

class DataLoader:
    def __init__(self, math23k_train, math23k_test):
        self.math23k_train_list = math23k_train
        self.math23k_test_list = math23k_test
        emb_dim = 128
        self.emb_vectors, self.vocab_list = self.preprocess_and_word2vec(emb_dim)
        self.vocab_dict = dict([(elem, idx) for idx, elem in enumerate(self.vocab_list)])
        self.vocab_len = len(self.vocab_list)

        self.classes_len = 2
        
    def preprocess_and_word2vec(self, emb_dim):
        new_data ={}
        sentences = []
        for elem in self.math23k_train_list:
            sentence = elem['new_split'].strip().split(' ')
            sentences.append(sentence)
            for elem in sentence:
                new_data[elem] = new_data.get(elem, 0) + 1
        
        from gensim.models import word2vec
        model = word2vec.Word2Vec(sentences, size=emb_dim, min_count=1)

        token_list = ['PAD_token', 'UNK_token']
        emb_vectors = []
        emb_vectors.append(np.zeros((emb_dim)))
        emb_vectors.append(np.random.rand((emb_dim))/1000.0)
        
        for k, v in new_data.items(): 
            token_list.append(k)
            emb_vectors.append(np.array(model.wv[k]))
        
        emb_vectors = np.array(emb_vectors)
        return emb_vectors, token_list

    def get_batch_train(self, data_set, batch_size):
        data_list = []
        label_list = []
        id_list = []
        for elem in data_set:
            context_list = elem['context_list'] 
            num_labels = elem['num_labels']
            id_ = elem['id']
            for c_i in range(len(context_list)):
                context = context_list[c_i]
                data_list.append(context)
                id_list.append([id_, c_i])
            for num in num_labels:
                label_list.append(num)
             
        data_set = list(zip(data_list, label_list, id_list))
        #print (type(data_set))
        random.seed(10)
        random.shuffle(data_set)
        
        batch_num = int(len(data_set)/batch_size)+1
        for idx in range(batch_num):
            batch_start = idx*batch_size
            batch_end = min((idx+1)*batch_size, len(data_set))
            
            data_batch = data_set[batch_start: batch_end]
            batch_x, batch_y, batch_ids = zip(*data_batch)
            
            batch_encode_idx = []
            batch_encode_len = []
            batch_encode_pad_idx = []
            
            for se_i in range(len(batch_x)):
                encode_sen = batch_x[se_i] # splitted
                encode_sen_idx = string_2_idx_sen(encode_sen, self.vocab_dict)
                encode_sen_pad_idx = pad_sen(encode_sen_idx, 7, self.vocab_dict['PAD_token']) 
                batch_encode_idx.append(encode_sen_idx)
                batch_encode_len.append(len(encode_sen_idx))
                batch_encode_pad_idx.append(encode_sen_pad_idx)
            
            batch_data_dict = dict()
            batch_data_dict['batch_encode_idx'] = batch_encode_idx
            batch_data_dict['batch_encode_pad_idx'] = batch_encode_pad_idx
            batch_data_dict['batch_encode_len'] = batch_encode_len 
            batch_data_dict['batch_y'] = batch_y
            batch_data_dict['batch_x'] = batch_x
            batch_data_dict['batch_ids'] = batch_ids
            
            new_batch_data_dict = self._sorted_batch(batch_data_dict)
            yield new_batch_data_dict#new_batch_data_dict

    def _sorted_batch(self, batch_data_dict):
        new_batch_data_dict = dict()
        batch_encode_len = np.array(batch_data_dict['batch_encode_len'])
        sort_idx = np.argsort(-batch_encode_len)
        for key, value in batch_data_dict.items():
            new_batch_data_dict[key] = np.array(value)[sort_idx]
        return new_batch_data_dict
    
            
                
            
            
            

        


In [9]:
#data_loader = DataLoader(math23k_train, math23k_test)
#train_generator = data_loader.get_batch_train(32)
#for elem in train_generator:
    #print (elem)
    #break

In [10]:
data_loader = DataLoader(math23k_train, math23k_test)

In [11]:
class BaseRNN(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, input_dropout_p, dropout_p, \
                          n_layers, rnn_cell_name):
        super(BaseRNN, self).__init__()
        self.vocab_size = vocab_size
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.input_dropout_p = input_dropout_p
        self.input_dropout = nn.Dropout(p=input_dropout_p)
        self.rnn_cell_name = rnn_cell_name
        if rnn_cell_name.lower() == 'lstm':
            self.rnn_cell = nn.LSTM
        elif rnn_cell_name.lower() == 'gru':
            self.rnn_cell = nn.GRU
        else:
            raise ValueError("Unsupported RNN Cell: {0}".format(rnn_cell_name))
        self.dropout_p = dropout_p
 
    def forward(self, *args, **kwargs):
        raise NotImplementedError()

class EncoderRNN(BaseRNN):
    def __init__(self, vocab_size, embed_model, emb_size=100, hidden_size=128, \
                 input_dropout_p=0, dropout_p=0, n_layers=1, bidirectional=False, \
                 rnn_cell=None, rnn_cell_name='gru', variable_lengths_flag=True):
        super(EncoderRNN, self).__init__(vocab_size, emb_size, hidden_size,
              input_dropout_p, dropout_p, n_layers, rnn_cell_name)
        self.variable_lengths_flag = variable_lengths_flag
        self.bidirectional = bidirectional
        self.embedding = embed_model
        if rnn_cell == None:
            self.rnn = self.rnn_cell(emb_size, hidden_size, n_layers,
                                 batch_first=True, bidirectional=bidirectional, dropout=dropout_p)
        else:
            self.rnn = rnn_cell
 
    def forward(self, input_var, input_lengths=None):
        embedded = self.embedding(input_var)
        embedded = self.input_dropout(embedded)
        #pdb.set_trace()
        if self.variable_lengths_flag:
            embedded = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, batch_first=True)
        output, hidden = self.rnn(embedded)
        if self.variable_lengths_flag:
            output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        return output, hidden
    
    def no_sorted_forward(self, input_var, seq_lengths=None):
        embedded = self.embedding(input_var)
        inputs = self.input_dropout(embedded)
        
        sorted_seq_lengths, indices = torch.sort(seq_lengths, descending=True)
        _, desorted_indices = torch.sort(indices, descending=False)
        inputs = inputs[indices]
        packed_inputs = nn.utils.rnn.pack_padded_sequence(inputs, sorted_seq_lengths.cpu().numpy(), batch_first=True)
        res, state = self.rnn(packed_inputs)
        padded_res, _ = nn.utils.rnn.pad_packed_sequence(res, batch_first=True)
        desorted_res = padded_res[desorted_indices]
        return desorted_res, state

def _cat_directions(h, bidirectional):
    if bidirectional:
        h = torch.cat([h[0:h.size(0):2], h[1:h.size(0):2]], 2)
    return h
    
class SNI_net(nn.Module):
    def __init__(self, data_loader, encode_params):
        super(SNI_net, self).__init__()
        
        self.embed_model =  nn.Embedding(data_loader.vocab_len, encode_params['emb_size'])
        self.encoder = EncoderRNN(vocab_size = data_loader.vocab_len,
                              embed_model = self.embed_model,
                              emb_size = encode_params['emb_size'],
                              hidden_size = encode_params['hidden_size'],
                              input_dropout_p = encode_params['input_dropout_p'],
                              dropout_p = encode_params['dropout_p'],
                              n_layers = encode_params['n_layers'],
                              bidirectional = encode_params['bidirectional'],
                              rnn_cell = encode_params['rnn_cell'],
                              rnn_cell_name = encode_params['rnn_cell_name'],
                              variable_lengths_flag = encode_params['variable_lengths_flag'])
        self.activation = F.relu
        self.encode_params = encode_params
        if encode_params['bidirectional'] == True:
            fnn_size = encode_params['hidden_size']*2
        else:
            fnn_size = encode_params['hidden_size']
        
        self.numClasses = data_loader.classes_len 
        self.data_loader = data_loader
        self.W = nn.Linear(fnn_size, fnn_size, bias=True)
        self.projection = nn.Linear(fnn_size, self.numClasses, bias=True)
        
    def forward(self, input_tensor, input_lengths):
        '''
        1--: torch.Size([32, 7])
        2--: torch.Size([2, 32, 128])
        3--: torch.Size([1, 32, 256])
        4--: torch.Size([1, 32, 2])
        '''
        #print ('1--:', input_tensor.size())
        #encoder_outputs, encoder_hidden = self.encoder.no_sorted_forward(input_tensor, input_lengths)
        encoder_outputs, encoder_hidden = self.encoder(input_tensor, input_lengths)
        #print ('2--:', encoder_hidden[0].size())
        encoder_hidden_ = _cat_directions(encoder_hidden[0], self.encode_params['bidirectional'])
        #print ('3--:', encoder_hidden_.size())
        fc_input = encoder_hidden_.squeeze(0)
        currentNode = self.activation(self.W(fc_input))
        proj_probs = self.projection(currentNode)
        #print ('4--:', proj_probs.size()) 
        return proj_probs
    
    
        

In [18]:
class Trainer():
    def __init__(self, data_loader, params, optimizer):
        self.data_loader = data_loader
        self.params = params
        self.optimizer = optimizer
    
    def train(self, model):
        for epoch in range(self.params['start_epoch'], self.params['n_epoch']):
            train_generator = self.data_loader.get_batch_train(self.data_loader.math23k_train_list, self.params['batch_size'])
            epoch_loss = 0.0#torch.FloatTensor([0])[0]#.cuda()
            pred_count = 0
            total = 0
            for batch_elem in train_generator:
                batch_y = batch_elem['batch_y']
                batch_encode_pad_idx = batch_elem['batch_encode_pad_idx']
                batch_encode_len = batch_elem['batch_encode_len']

                batch_encode_pad_idx_tensor = torch.LongTensor(batch_encode_pad_idx).cuda()
                batch_encode_len_tensor = torch.LongTensor(batch_encode_len).cuda()
                batch_y_tensor = torch.LongTensor(batch_y).cuda()
                
                proj_probs = model(batch_encode_pad_idx_tensor, batch_encode_len_tensor)
                #max_probs = proj_probs.max(dim=1)[1]
                
                loss = F.cross_entropy(input=proj_probs, target=batch_y_tensor)
                
                self.optimizer.zero_grad()
                
                loss.backward()#(retain_graph=True)
                clip_grad_norm_(model.parameters(), 5, norm_type=2.)
                self.optimizer.step()
                
                epoch_loss += loss.item()
                
                batch_preds = proj_probs.max(dim=1)[1]
                
                pred_count += torch.sum((batch_preds == batch_y_tensor).int()).item()
                total += len(batch_y)
            print ("epoch:", epoch, 'loss:', epoch_loss*1.0/total, 'train_acc:', pred_count*1.0/total)
             
            train_generator = self.data_loader.get_batch_train(self.data_loader.math23k_train_list, self.params['batch_size'])
            train_accumulate_dict = self.predict_mid(model, train_generator)
            print ("i_train_acc:", self.predict_acc(self.data_loader.math23k_train_list, train_accumulate_dict))
            test_generator = self.data_loader.get_batch_train(self.data_loader.math23k_test_list, self.params['batch_size'])
            test_accumulate_dict = self.predict_mid(model, test_generator)
            print ("i_test_acc:", self.predict_acc(self.data_loader.math23k_test_list, test_accumulate_dict))
            print ()
        
    def predict_acc(self, data_list, accumulate_dict):
        acc = 0.0
        for elem in data_list:
            context_list = elem['context_list'] 
            num_labels = elem['num_labels']
            id_ = elem['id']
            flag = 1
            for c_i in range(len(context_list)):
                context = context_list[c_i]
                label_ = num_labels[c_i]
                k_ = str(id_) + '_' + str(c_i)
                if label_ != accumulate_dict[k_]:
                    flag = 0
            acc += flag
        return acc*1.0/len(data_list)
                
            
    
    def predict_mid(self, model, data_generator):
        accumulate_dict = {}
        for batch_elem in data_generator:
            batch_y = batch_elem['batch_y']
            batch_encode_pad_idx = batch_elem['batch_encode_pad_idx']
            batch_encode_len = batch_elem['batch_encode_len']

            batch_encode_pad_idx_tensor = torch.LongTensor(batch_encode_pad_idx).cuda()
            batch_encode_len_tensor = torch.LongTensor(batch_encode_len).cuda()
            batch_y_tensor = torch.LongTensor(batch_y).cuda()

            proj_probs = model(batch_encode_pad_idx_tensor, batch_encode_len_tensor)
            
            batch_preds = proj_probs.max(dim=1)[1]
            
            batch_ids = batch_elem['batch_ids']
            
            for b_i in range(len(batch_y)):
                id_ = batch_ids[b_i]
                p_ = batch_preds[b_i].item()
                accumulate_dict['_'.join(id_)] = p_
        return accumulate_dict
            
        
            

In [19]:
params = {"batch_size": 32,
          "start_epoch" : 1,
          "n_epoch": 150,
          "rnn_classes":5,
          "save_file": "model_xxx_att.pkl"
         }
encode_params = {
    "emb_size":  128,
    "hidden_size": 128,
    "input_dropout_p": 0.2,
    "dropout_p": 0.3,
    "n_layers": 1,
    "bidirectional": True,
    "rnn_cell": None,
    "rnn_cell_name": 'lstm',
    "variable_lengths_flag": True
}

sni_net = SNI_net(data_loader, encode_params).cuda()

optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, sni_net.parameters()), \
                            lr=0.01, momentum=0.9, dampening=0.0)

trainer = Trainer(data_loader, params, optimizer)
trainer.train(sni_net)


/home/wanglei/.pyvenvs/py35/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


epoch: 1 loss: 0.005119848604746655 train_acc: 0.9544065165554435
i_train_acc: 0.899557801642451
i_test_acc: 0.875

epoch: 2 loss: 0.004385098702973675 train_acc: 0.9580232688399187
i_train_acc: 0.9072737117588665
i_test_acc: 0.884

epoch: 3 loss: 0.004034018811841637 train_acc: 0.9613999711940083
i_train_acc: 0.9133200974641278
i_test_acc: 0.89

epoch: 4 loss: 0.0037771988537166775 train_acc: 0.9632883639797077
i_train_acc: 0.9169750022561141
i_test_acc: 0.893

epoch: 5 loss: 0.0035903986722804664 train_acc: 0.964776673548098
i_train_acc: 0.918779893511416
i_test_acc: 0.891

epoch: 6 loss: 0.003430682936739963 train_acc: 0.9659769232000256
i_train_acc: 0.9204945402039527
i_test_acc: 0.899

epoch: 7 loss: 0.0032999439851993167 train_acc: 0.9667610863059516
i_train_acc: 0.923246999368288
i_test_acc: 0.904

epoch: 8 loss: 0.0031652328617838496 train_acc: 0.9676732760414166
i_train_acc: 0.9252323797491201
i_test_acc: 0.898

epoch: 9 loss: 0.0030259740918014825 train_acc: 0.969033558980267

i_train_acc: 0.9694070932226333
i_test_acc: 0.903

epoch: 71 loss: 0.000985603868815988 train_acc: 0.9874533903051835
i_train_acc: 0.9697229491923112
i_test_acc: 0.902

epoch: 72 loss: 0.0009632373882967169 train_acc: 0.9876294269207996
i_train_acc: 0.9701290497247541
i_test_acc: 0.897

epoch: 73 loss: 0.0009624543700835266 train_acc: 0.9877094435642614
i_train_acc: 0.9702192942875192
i_test_acc: 0.898

epoch: 74 loss: 0.0009458808051188875 train_acc: 0.9879494934946469
i_train_acc: 0.969046114971573
i_test_acc: 0.902

epoch: 75 loss: 0.0009655061252713861 train_acc: 0.9877734568790308
i_train_acc: 0.9696327046295461
i_test_acc: 0.899

epoch: 76 loss: 0.0009563460167264622 train_acc: 0.987821466865108
i_train_acc: 0.969046114971573
i_test_acc: 0.891

epoch: 77 loss: 0.0009382460607752435 train_acc: 0.9879014835085698
i_train_acc: 0.9703997834130493
i_test_acc: 0.899

epoch: 78 loss: 0.0009453238196207665 train_acc: 0.9877414502216461
i_train_acc: 0.9704449056944319
i_test_acc: 0.903

e

i_train_acc: 0.9713473513220828
i_test_acc: 0.901

epoch: 140 loss: 0.0008409299748511901 train_acc: 0.9886376366284187
i_train_acc: 0.9714827181662304
i_test_acc: 0.902

epoch: 141 loss: 0.0008471011875249769 train_acc: 0.988605629971034
i_train_acc: 0.9717985741359083
i_test_acc: 0.896

epoch: 142 loss: 0.0008505402256498916 train_acc: 0.9884295933554179
i_train_acc: 0.9718888186986734
i_test_acc: 0.9

epoch: 143 loss: 0.0008474116891236942 train_acc: 0.9883495767119561
i_train_acc: 0.9724302860752639
i_test_acc: 0.896

epoch: 144 loss: 0.000840722400811974 train_acc: 0.9885576199849568
i_train_acc: 0.9716632072917607
i_test_acc: 0.904

epoch: 145 loss: 0.0008475731486977455 train_acc: 0.9883335733832637
i_train_acc: 0.9716632072917607
i_test_acc: 0.906

epoch: 146 loss: 0.0008337691031302826 train_acc: 0.9889416998735737
i_train_acc: 0.9714827181662304
i_test_acc: 0.909

epoch: 147 loss: 0.0008408823107152624 train_acc: 0.9886376366284187
i_train_acc: 0.9719339409800559
i_test_acc: 